In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly import graph_objects as go
from WindPy import w
import datetime
from sqlalchemy import create_engine, text

In [45]:
w.start()
print(w.isconnected())

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
True


In [2]:
engine = create_engine(
    "mysql+pymysql://",
    connect_args={
        "host": "172.16.16.33",
        "user": "hry",
        "password": "hry@8888",  # Password with @ goes here
        "database": "ry",
        "port": 3306,
        "charset": "utf8"
    }
)

In [3]:
def previous_trading_day(date):
    # If today is Monday, move to Friday
    if date.weekday() == 0:
        return date - datetime.timedelta(days=3)
    # If today is Sunday, move to Friday
    elif date.weekday() == 6:
        return date - datetime.timedelta(days=2)
    # For weekdays (Monday to Friday), just go back one day
    else:
        return date - datetime.timedelta(days=1)

In [62]:
today = datetime.date.today().strftime('%Y-%m-%d')
last_trading_day = previous_trading_day(datetime.date.today()).strftime('%Y-%m-%d')
# last_trading_day_str = last_trading_day.strftime('%Y-%m-%d')

In [63]:
today

'2025-09-30'

In [64]:
last_trading_day

'2025-09-29'

In [34]:
anyday = datetime.datetime(2025,9,26)

In [65]:
data = pd.read_sql(f"SELECT * FROM index_data", engine)
# data['date'] = pd.to_datetime(data['date'])
# pd.reset_option('display.max_columns')

In [66]:
newdata = w.wss("000300.SH,000852.SH,399373.SZ,399376.SZ,881001.WI", "close,pb_lf",f"tradeDate={last_trading_day};priceAdj=U;cycle=D").Data
newdata_list = newdata[0] + newdata[1]
newdata_list = [round(x, 4) for x in newdata_list]
data.iloc[len(data)-1,1:11] = newdata_list
data.loc[len(data), 'date'] = today
data['year'] = data['date'].str[:4]

In [67]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,...,CSI300_Close_2,CSI1000_Close_2,CSI300_rtn_1,CSI1000_rtn_1,Signal4,Signal4_day_rtn,Position4,Signal_5,Signal5_day_rtn,Position5
0,2010-01-04,3535.2290,4406.7650,4801.7380,4111.9040,2917.8525,3.3084,NaN,2.7787,4.3815,...,3558.8570,4336.6050,1.004728,1.012608,-1.0,1.003526,满仓中证1000指数,-4.0,1.011130,7.5%多IM + 7.5%空IF
1,2010-01-05,3564.0380,4462.2320,4829.1060,4158.2700,2941.4729,3.3449,NaN,2.8017,4.4299,...,3575.6840,4391.2810,0.988686,1.003526,-1.0,1.012587,满仓中证1000指数,-4.0,1.003328,7.5%多IM + 7.5%空IF
2,2010-01-06,3541.7270,4452.8680,4780.3700,4153.2210,2921.8748,3.3133,NaN,2.7692,4.3964,...,3535.2290,4406.7650,1.008149,1.012587,-1.0,0.997901,满仓中证1000指数,-4.0,1.003121,7.5%多IM + 7.5%空IF
3,2010-01-07,3471.4560,4361.1610,4673.6460,4072.9810,2860.3718,3.2496,NaN,2.7060,4.3115,...,3564.0380,4462.2320,0.993740,0.997901,-1.0,0.979405,满仓中证1000指数,-4.0,0.999434,7.5%多IM + 7.5%空IF
4,2010-01-08,3480.1300,4417.7710,4683.2360,4126.6380,2872.4092,3.2475,NaN,2.7084,4.3647,...,3541.7270,4452.8680,0.980159,0.979405,0.0,1.000000,空仓,-2.0,1.007861,7.5%多IM + 7.5%空IF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,2025-09-24,4566.0708,7534.2223,8074.9849,6194.5181,6289.6762,1.4645,2.5297,0.8992,3.2684,...,4522.6080,7489.4784,0.999374,0.989130,1.0,1.010242,满仓沪深300指数,0.0,1.000000,空仓
3822,2025-09-25,4593.4875,7506.5124,8029.1614,6191.4151,6300.6440,1.4684,2.5200,0.8954,3.2634,...,4519.7781,7408.0691,1.010242,1.017029,-1.0,0.996322,满仓中证1000指数,-4.0,0.992738,7.5%多IM + 7.5%空IF
3823,2025-09-26,4550.0487,7397.5894,8044.0930,6079.5242,6224.9086,1.4590,2.4861,0.8957,3.2136,...,4566.0708,7534.2223,1.006004,0.996322,1.0,0.990543,满仓沪深300指数,-1.0,0.997473,5%多IM + 5%空IF
3824,2025-09-29,4620.0528,7497.8308,8074.7172,6187.8720,6310.0606,1.4738,2.5182,0.8946,3.2684,...,4593.4875,7506.5124,0.990543,0.985490,1.0,NaN,满仓沪深300指数,2.0,NaN,5%多IF + 5%空IM


#### **动量因子风格择时信号：**

##### 需要数据： 沪深300指数和中证1000指数日收盘价 
##### 信号逻辑： 取前第一日和第十一日指数收盘价计算过去十天指数收益率：
#####        (1) 沪深300指数收益率 > 中证1000指数收益率： 当天开盘买入沪深300指数,收盘平仓
#####        (2) 沪深300指数收益率 < 中证1000指数收益率： 当天开盘买入中证1000指数，收盘平仓
#####        (3) 若两者收益率均小于0：当天空仓

In [68]:
data["CSI300_Close_1"] = data['CSI300_Close'].shift()
data["CSI300_Close_11"] = data['CSI300_Close'].shift(11)
data["CSI1000_Close_1"] = data['CSI1000_Close'].shift()
data["CSI1000_Close_11"] = data['CSI1000_Close'].shift(11)
data['10_days_CSI300_rtn'] = data['CSI300_Close_1']/data['CSI300_Close_11']-1
data['10_days_CSI1000_rtn'] = data['CSI1000_Close_1']/data['CSI1000_Close_11']-1

In [69]:
def signal1_row(row):
    if (row['10_days_CSI300_rtn'] * row['10_days_CSI1000_rtn'] > 0) & (row['10_days_CSI300_rtn'] < 0):
        row['Signal1'] = 0
        row['Signal1_day_rtn'] = 1
        row['Position1'] = '空仓'
    elif row['10_days_CSI300_rtn'] > row['10_days_CSI1000_rtn']:
        row['Signal1'] = 1
        row['Signal1_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position1'] = '满仓沪深300指数'
    else: 
        row['Signal1'] = -1
        row['Signal1_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position1'] = '满仓中证1000指数'
    return row

data = data.apply(signal1_row, axis = 1)

In [174]:
# fig1 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal1_rtn'],
#                  text = data['Position1'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig1.update_layout(template='simple_white',
#                    title='<b>动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig1.show()

#### **拥挤度动量因子风格择时信号：**

##### 需要数据： 大盘价值指数PB,小盘成长指数PB，万得全A指数PB 
##### 信号逻辑： 取前第一日和第五日指数PB计算指标：
#####           R1 = (大盘价值指数PB-小盘成长指数PB)/万得全A指数PB
#####        (1) R1(t-1) > R1(t-5)： 当天开盘买入沪深300指数,收盘平仓
#####        (2) R1(t-1) < R1(t-5)： 当天开盘买入中证1000指数，收盘平仓

In [70]:
data['LCVI_PB_1'] = data['LCVI_PB'].shift()
data['LCVI_PB_5'] = data['LCVI_PB'].shift(5)
data['SCGI_PB_1'] = data['SCGI_PB'].shift()
data['SCGI_PB_5'] = data['SCGI_PB'].shift(5)
data['WASI_PB_1'] = data['WASI_PB'].shift()
data['WASI_PB_5'] = data['WASI_PB'].shift(5)

In [71]:
def signal2_row(row):
    if (row['LCVI_PB_1'] - row['SCGI_PB_1'])/row['WASI_PB_1'] > (row['LCVI_PB_5'] - row['SCGI_PB_5'])/row['WASI_PB_5']:
        row['Signal2'] = 1
        row['Signal2_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position2'] = '满仓沪深300指数'
    else: 
        row['Signal2'] = -1
        row['Signal2_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position2'] = '满仓中证1000指数'
    return row

data = data.apply(signal2_row, axis = 1)

In [333]:
# fig2 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal2_rtn'],
#                  text = data['Position2'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig2.update_layout(template='simple_white',
#                    title='<b>拥挤度动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081",
#                    ),
#                    width=2000, 
#                    height=500)
# fig2.show()

#### **相对强弱动量因子风格择时信号（大盘价值指数和小盘成长指数）**

##### 需要数据： 大盘价值指数、小盘成长指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 大盘价值指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 小盘成长指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [72]:
data['LCVI_Close_1'] = data['LCVI_Close'].shift()
data['LCVI_Close_2'] = data['LCVI_Close'].shift(2)
data['SCGI_Close_1'] = data['SCGI_Close'].shift()
data['SCGI_Close_2'] = data['SCGI_Close'].shift(2)
data['WASI_Close_1'] = data['WASI_Close'].shift()
data['WASI_Close_2'] = data['WASI_Close'].shift(2)
data['LCVI_rtn_1'] = data['LCVI_Close_1']/data['LCVI_Close_2']
data['SCGI_rtn_1'] = data['SCGI_Close_1']/data['SCGI_Close_2']
data['WASI_rtn_1'] = data['WASI_Close_1']/data['WASI_Close_2']

In [73]:
def signal3_row(row):
    if (row['LCVI_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['SCGI_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal3'] = 0
        row['Signal3_day_rtn'] = 1
        row['Position3'] = '空仓'
    elif row['LCVI_rtn_1'] > row['SCGI_rtn_1']:
        row['Signal3'] = 1
        row['Signal3_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position3'] = '满仓沪深300指数'
    else: 
        row['Signal3'] = -1
        row['Signal3_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position3'] = '满仓中证1000指数'
    return row

data = data.apply(signal3_row, axis = 1)

In [180]:
# fig3 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal3_rtn'],
#                  text = data['Position3'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig3.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(大盘价值指数和小盘成长指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig3.show()

#### **相对强弱动量因子风格择时信号（沪深300指数和中证1000指数）**

##### 需要数据： 沪深300指数、中证1000指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 沪深300指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 中证1000指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [74]:
data['CSI300_Close_2'] = data['CSI300_Close'].shift(2)
data['CSI1000_Close_2'] = data['CSI1000_Close'].shift(2)
data['CSI300_rtn_1'] = data['CSI300_Close_1']/data['CSI300_Close_2']
data['CSI1000_rtn_1'] = data['CSI1000_Close_1']/data['CSI1000_Close_2']

In [75]:
def signal4_row(row):
    if (row['CSI300_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['CSI1000_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal4'] = 0
        row['Signal4_day_rtn'] = 1
        row['Position4'] = '空仓'
    elif row['CSI300_rtn_1'] > row['CSI1000_rtn_1']:
        row['Signal4'] = 1
        row['Signal4_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position4'] = '满仓沪深300指数'
    else: 
        row['Signal4'] = -1
        row['Signal4_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position4'] = '满仓中证1000指数'
    return row

data = data.apply(signal4_row, axis = 1)

In [183]:
# fig4 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal4_rtn'],
#                  text = data['Position4'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig4.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(沪深300指数和中证1000指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig4.show()

#### **多维动量复合风格择时对冲信号**

##### 信号逻辑： 根据以上四个信号计算指标
#####          c1 = 择时沪深300指数信号数量
#####          c2 = 择时中证1000指数信号数量
#####         (1) c1 - c2 > 2: 7.5%多IF + 7.5%空IM
#####         (2) 0 < c1 - c2 < 2: 5%多IF + 5%空IM
#####         (3) c1 - c2 = 0: 空仓
#####         (4) -2 < c1 - c2 < 0: 5%多IM + 5%空IF
#####         (5) c1 - c2 < -2: 7.5%多IM + 7.5%空IF

In [76]:
margin = 0.10
data['Signal_5'] = data['Signal1'] + data['Signal2'] + data['Signal3'] + data['Signal4']

In [77]:
def signal5_row(row):
    if (row['Signal_5'] > 2):
        row['Signal5_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IF + 7.5%空IM'
    elif row['Signal_5'] > 0:
        row['Signal5_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IF + 5%空IM'
    elif row['Signal_5'] == 0:
        row['Signal5_day_rtn'] = 1
        row['Position5'] = '空仓'
    elif row['Signal_5'] > -2:
        row['Signal5_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IM + 5%空IF'
    else:
        row['Signal5_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IM + 7.5%空IF'
    return row

data = data.apply(signal5_row, axis = 1)

In [186]:
# fig5 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal5_rtn'],
#                  text = data['Position5'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig5.update_layout(template='simple_white',
#                    title='<b>多维动量复合风格择时对冲信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig5.show()

In [83]:
datanew = data.loc[data['date'] >= last_trading_day]

In [84]:
datanew

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,...,CSI300_Close_2,CSI1000_Close_2,CSI300_rtn_1,CSI1000_rtn_1,Signal4,Signal4_day_rtn,Position4,Signal_5,Signal5_day_rtn,Position5
3824,2025-09-29,4620.0528,7497.8308,8074.7172,6187.872,6310.0606,1.4738,2.5182,0.8946,3.2684,...,4593.4875,7506.5124,0.990543,0.985490,1,1.015385,满仓沪深300指数,2,1.000917,5%多IF + 5%空IM
3825,2025-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4550.0487,7397.5894,1.015385,1.013551,0,1.000000,空仓,-1,NaN,5%多IM + 5%空IF


In [85]:
with engine.connect() as conn:
    result = conn.execute(
        text("DELETE FROM index_data WHERE date = :date"),
        {'date': last_trading_day}
    )
    conn.commit()

In [86]:
datanew.to_sql('index_data', engine, if_exists='append', index=False)

2